In [171]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd

In [172]:
data = pd.read_csv('data/train-balanced-sarcasm.csv')
data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [173]:
df = data[['label', 'comment']].dropna()
df

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.
...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...
1010822,1,"whatever you do, don't vote green!"
1010823,1,Perhaps this is an atheist conspiracy to make ...
1010824,1,The Slavs got their own country - it is called...


In [174]:
comments = df['comment'].to_numpy()
targets = df['label'].to_numpy()

comments = np.array(comments)
targets = np.array(targets)

In [175]:
token = Tokenizer()
token.fit_on_texts(comments)
word_index=token.word_index
len(word_index)

180583

In [176]:
def word_to_seq(s):
    return tf.keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences([s]), maxlen=maxlen)

In [177]:
maxlen=max([len(i.split()) for i in comments])
print(maxlen)
train_data = word_to_seq(comments.tolist())
input_size = len(train_data)
train_data = np.array(train_data)
total=len(word_index)
print(train_data.shape[1])

2222
2222


In [182]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(total, 128, input_length=maxlen),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
        tf.keras.layers.Dropout(0.4), 
        tf.keras.layers.Dense(512, activation='softmax'),
        tf.keras.layers.Dropout(0.4), 
        tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 2222, 128)         23114624  
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 512)               788480    
_________________________________________________________________
dropout_28 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_29 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 1026      
Total params: 24,166,786
Trainable params: 24,166,786
Non-trainable params: 0
_________________________________________

In [183]:

history=model.fit(train_data[0], targets, epochs=10, validation_split=0.2)

Epoch 1/10
56/56 [==============================] - 17s 301ms/step - loss: 0.6478 - accuracy: 0.9111 - val_loss: 0.6122 - val_accuracy: 0.8697
Epoch 2/10
56/56 [==============================] - 16s 291ms/step - loss: 0.5496 - accuracy: 0.9128 - val_loss: 0.5203 - val_accuracy: 0.8697
Epoch 3/10
56/56 [==============================] - 16s 288ms/step - loss: 0.4413 - accuracy: 0.9128 - val_loss: 0.4608 - val_accuracy: 0.8697
Epoch 4/10
56/56 [==============================] - 17s 296ms/step - loss: 0.3639 - accuracy: 0.9128 - val_loss: 0.4332 - val_accuracy: 0.8697
Epoch 5/10
56/56 [==============================] - 17s 297ms/step - loss: 0.3201 - accuracy: 0.9128 - val_loss: 0.4200 - val_accuracy: 0.8697
Epoch 6/10
56/56 [==============================] - 16s 287ms/step - loss: 0.2910 - accuracy: 0.9145 - val_loss: 0.4136 - val_accuracy: 0.8697
Epoch 7/10
56/56 [==============================] - 17s 295ms/step - loss: 0.2635 - accuracy: 0.9415 - val_loss: 0.4099 - val_accuracy: 0.8517